In [1]:
import sys
sys.path.insert(0, '../ModeTonicRecognition/')

import json
import os
import matplotlib.pyplot as plt
from extras import foldGeneration
from extras import fileOperations as fo
import numpy as np
import ChordiaEstimation
import Evaluator as ev
from sklearn import cross_validation


In [2]:
# I/O
experiment_dir = data_dir = '../../experiments/raag-recognition/'
data_dir = os.path.join(experiment_dir,'data')
modes = fo.getModeNames(data_dir)

fold_savefile = os.path.join(experiment_dir, 'folds.json')
train_savefolder = os.path.join(experiment_dir, 'models')
results_savefolder = os.path.join(experiment_dir, 'results')

In [3]:
# instantiate objects
evaluator = ev.Evaluator()
che = ChordiaEstimation.ChordiaEstimation(step_size=10, smooth_factor=15, 
                                          chunk_size=120, threshold=0.5, 
                                          overlap=0, frame_rate=128.0/44100)


In [4]:
# get the data into appropriate format
[pitch_paths, pitch_base, pitch_fname] = fo.getFileNamesInDir(data_dir, '.pitch')
tonic_paths = [os.path.splitext(p)[0] + '.tonic' for p in pitch_paths]
mode_labels = []
for p in pitch_base:
    for r in modes:
        if r in p:
            mode_labels.append(r)
            

In [5]:
# make the data a single dictionary for housekeeping
data = []
for p, f, t, r in zip(pitch_paths, pitch_fname, tonic_paths, mode_labels):
    data.append({'file':p, 'name':os.path.splitext(f)[0],
               'tonic':float(np.loadtxt(t)), 'mode':r})


In [6]:
# stratified 14 fold 
mode_idx = [modes.index(m) for m in [d['mode'] for d in data]]
skf = cross_validation.StratifiedKFold(mode_idx, n_folds=14)

folds = dict()
for ff, fold in enumerate(skf):
    folds['fold' + str(ff)] = {'train': [], 'test': []}
    for tr_idx in fold[0]:
        folds['fold' + str(ff)]['train'].append(data[tr_idx])
    for te_idx in fold[1]:
        folds['fold' + str(ff)]['test'].append(data[te_idx])
        
with open(fold_savefile, 'w') as f:
    json.dump(folds, f, indent=2)

In [7]:
# experiments
results = dict()
for key, fold in folds.iteritems():
    # Training 
    print key
    models = dict()
    for cur_mode in modes:
        [file_list, tonic_list] = zip(*[(rec['file'], rec['tonic']) for rec in fold['train']
                                        if rec['mode'] == cur_mode])
        models[cur_mode] = che.train(cur_mode, file_list, tonic_list, metric='pcd', 
                                     save_dir = os.path.join(train_savefolder, key))
                                     
    # Raag Recognition
    results[key] = []
    for rec in fold['test']:
        rec['pitch'] = np.loadtxt(rec['file'])[:,1]
        res = che.estimate(rec['pitch'], mode_names=modes, 
                           mode_dir=os.path.join(train_savefolder, key), 
                           est_tonic=False, est_mode=True, distance_method='bhat',
                           metric='pcd', ref_freq=rec['tonic'])[0]

        # evaluate
        results[key].append(evaluator.mode_evaluate(rec['file'], res, rec['mode']))

    print key + ": " + str(100*np.mean([r['mode_eval'] for r in results[key]])) + '%'
    

fold8
fold8: 80.0%
fold9
fold9: 100.0%
fold2
fold2: 90.0%
fold3
fold3: 100.0%
fold0
fold0: 80.0%
fold1
fold1: 70.0%
fold6
fold6: 80.0%
fold7


KeyboardInterrupt: 

In [8]:
results

{'fold0': [{'annotated_mode': '09c179f3-8b19-4792-a852-e9fa0090e409',
   'estimated_mode': '09c179f3-8b19-4792-a852-e9fa0090e409',
   'mbid': '../../experiments/raag-recognition/data/09c179f3-8b19-4792-a852-e9fa0090e409/Amritha_Murali/December_Season_2011/Brindavana_Dolagaaduva/Brindavana_Dolagaaduva.pitch',
   'mode_eval': True},
  {'annotated_mode': '123b09bd-9901-4e64-a65a-10b02c9e0597',
   'estimated_mode': '123b09bd-9901-4e64-a65a-10b02c9e0597',
   'mbid': '../../experiments/raag-recognition/data/123b09bd-9901-4e64-a65a-10b02c9e0597/Abhishek_Raghuram/December_Season_2010/Viriboni/Viriboni.pitch',
   'mode_eval': True},
  {'annotated_mode': '700e1d92-7094-4c21-8a3b-d7b60c550edf',
   'estimated_mode': '700e1d92-7094-4c21-8a3b-d7b60c550edf',
   'mbid': '../../experiments/raag-recognition/data/700e1d92-7094-4c21-8a3b-d7b60c550edf/Amrutha_Venkatesh/December_Season_2010/Narayana_Te_Namo___/Narayana_Te_Namo___.pitch',
   'mode_eval': True},
  {'annotated_mode': '77bc3583-d798-4077-85a3-b